In [1]:
%matplotlib inline
import matplotlib

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import discopt
import pandas as pd
import numpy as np
import copy
import sklearn
import sklearn.datasets
import xgboost as xgb
import scipy
from scipy import stats

In [4]:
n_jobs = 3
ntrials = 30

# Prepare data 

In [5]:
np.random.seed(0)
digits = sklearn.datasets.load_digits()
x = digits.data
y = digits.target

ntrain = 100
order = np.random.permutation(y.size)
tr = order[:ntrain]
val = order[ntrain:]
ytr = y[tr]
xtr = x[tr, :]
yval = y[val]
xval = x[val, :]


# DISCOPT SHORTCUT FOR XGB 

In [6]:
# parameter space is proposed in discopt.py file

In [6]:
# write your train_and_valid function
def train_and_valid(clf):
    # put here trainig and validation, return objective
    # optimizer would try to minimize this value, so use NEGATIVE accuracy/AUC/F1...
    # validation method is up to you
    # clf is xgboost.sklearn.XGBoostClassifier instance
    clf.fit(xtr, ytr)
    pred = clf.predict(xval)
    obj = -(pred == yval).mean() # (minus) accuracy
    return obj

In [7]:
best_params, scores = discopt.search_params_for_xgb(train_valid_func=train_and_valid,
                                                    train_data_shape=xtr.shape,
                                                    ntrials=ntrials,
                                                    n_jobs=n_jobs)

best objective yet -0.852681202121
best objective yet -0.852681202121
best objective yet -0.852681202121
best objective yet -0.852681202121
best objective yet -0.852681202121
best objective yet -0.852681202121
best objective yet -0.852681202121
best objective yet -0.855627578079
best objective yet -0.855627578079
best objective yet -0.855627578079
best objective yet -0.855627578079
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.873895109016
best objective yet -0.878609310548
best objective yet -0.878609310548
best objective yet -

In [13]:
# the long 'stagnations' of 'best value so far' is typical for every optimizer, at least for this particular problem

In [8]:
best_params

{'colsample_bylevel': 0.99286728518736778,
 'colsample_bytree': 0.23387603400036899,
 'gamma': 0.028237252232474395,
 'learning_rate': 0.02381313753564103,
 'max_depth': 14,
 'n_estimators': 220,
 'reg_lambda': 0.043434594754154628,
 'subsample': 0.84847829800004337}

In [9]:
# dataframe with objective and 
scores.head()

,lr_trees_ratio,colsample_bytree,colsample_bylevel,max_depth,n_estimators,subsample,log_reg_lambda,log_gamma,obj
0,6.960429,0.667927,0.324535,11.0,259.0,0.213756,1.866324,-2.268405,-0.704773
1,3.235117,0.260937,0.359311,5.0,290.0,0.937539,-2.027846,-2.073616,-0.852681
2,5.526780,0.511301,0.910028,5.0,161.0,0.380002,-2.184782,1.380114,-0.748379
3,6.419523,0.608833,0.619484,20.0,248.0,0.357533,-1.358569,0.193842,-0.834414
4,3.822966,0.325159,0.235959,13.0,56.0,0.475824,-1.882403,-4.482429,-0.817914
